In [1]:
!pip install pandas joblib



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
import os

sys.path.append(os.path.abspath("../"))  


In [3]:
import pandas as pd
import joblib
from src.preprocessing import clean_text
from src.model import EmotionClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
exclude_keywords = [
    'technology & engineering', 'women\'s health', 'military science',
    'health & fitness', 'diet & nutrition', 'cooking', 'reference',
    'cookbook', 'recipe', 'cooking', 'manual', 'how to', 'business',
    'marketing', 'guide', 'biography', 'memoir', 'family & relationships',
    'life stages', 'teenagers', 'religion', 'general', 'christian life'
]
def is_relevant(category):
    category = str(category).lower()  
    return not any(keyword in category for keyword in exclude_keywords)

In [5]:
books = pd.read_csv('../data/books.csv')
books = books[['Title', 'Authors', 'Description','Category']].dropna()
books = books[books['Description'].str.strip() != ""]
books = books[books['Category'].apply(is_relevant)]

In [6]:
model = joblib.load('../outputs/emotion_model.pkl')

In [7]:
emotion_dict = {
    0: 'Sadness',
    1: 'Joy',
    2: 'Love',
    3: 'Anger',
    4: 'Fear',
    5: 'Surprise'
}


In [8]:
def detect_emotion(text):
    cleaned = clean_text(text)
    pred = model.predict([cleaned])[0]
    return emotion_dict[int(pred)]

In [9]:
def recommend_books(emotion_label, books_df, top_n=5):
    recommendation_map = {
        'Sadness': 'Joy',
        'Joy': 'Joy',
        'Love': 'Love',
        'Anger': 'Calm',
        'Fear': 'Inspiration',
        'Surprise': 'Adventure'
    }

# Get the target vibe for the emotion based on the recommendation map.
# If no mapping is found for the given emotion, it will return the emotion itself as the default.
    target_vibe = recommendation_map.get(emotion_label, emotion_label)
    filtered = books_df[books_df['Description'].str.contains(target_vibe, case=False, na=False)]

    return filtered.sample(top_n) if not filtered.empty else books_df.sample(top_n)

In [14]:
user_input = "I'm in such a great mood today, and I just want to read something light and uplifting"
detected_emotion = detect_emotion(user_input)
print("Detected Emotion:", detected_emotion)

recommendations = recommend_books(detected_emotion, books)
print("\n Book Recommendations:")
print(recommendations[['Title', 'Authors']])

Detected Emotion: Joy

 Book Recommendations:
                                                   Title  \
45723                                 The Joy of Origami   
86194                                            Tainted   
72384   We Are Their Heaven: Why the Dead Never Leave Us   
34071     Fluffy's Thanksgiving (level 3) (Hello Reader)   
90005  Excursions in Number Theory (Dover Books on Ma...   

                                        Authors  
45723                  By Van Sicklen, Margaret  
86194                         By Morgan, Brooke  
72384                        By DuBois, Allison  
34071  By Smith, Mavis (ILT) and McMullan, Kate  
90005                     By Ogilvy, C. Stanley  
